# Chapter 2

In this chapter, we'll cover the following recipes:
- Preparing a task
- Setting up a cron job
- Capturing errors and problems
- Sending email notifications

We'll begin by going over how we ought to prepare a task before we automate it.

### Preparing a task

It all starts with defining precisely the woek that needs to be executed, and designing it in a way that deosn't require human intervention to run.

Some ideal characteristic points are as follows:

1. **Sinble, clear entry point:** No confusion on how to start the task.
2. **Clear parameters:** If there are any parameters, they should be as explicit as possible.
3. **No interactivity:** Stopping the execution to request information from the user is not possible.
4. **The result should be stored:** In order to be checked at a different time than when it runs.
5. **Clear result:** When we oversee the execution of a program ourselves, we can accept more verbose results, such as unlabeled data or extra debugging information. However, for an automated task, the final result should be as concise and to the point as possible.
6. **Errors should be logged:** To analyze what went wrong.

A command-line programs has a lot of those characteristics already. It always has a clear entry point, with defined parameters, and the result can be stored, even if just in text format. And it can be improves ensuring a config file that clarifies the parameters, and an output file.

Not that point 6 is the objective of the *Capturing errors and problems* recipe, and will be covered there. 

To avoid interactivity, do not use any function that waits for user input, such as *input*. Remeber to delete debugger breakpoints!.


### Getting ready

We'll start by following a structure in which a main function will serve as the entry point, and all parameters are supplied to it.

This is the same basic structure that was presented in the *Adding command-line arguments* recipe in *Chapter 1, Let's Begin Out Automation Journey.*

### How to do it...
1. Prepare the following command-line  program by multiplying two numbers, and save it as `prepare_task_step1.py`

In [ ]:
import argparse

def main(number, other_number):
    result = number * other_number
    print(f'The result is {result}')
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-n1', type=int, help= 'A number', default=1)
    parser.add_argument('-n2', type=int, help='Another number', default=1)
    
    args=parser.parse_args()
    main(args.n1, args.n2)

2. Update the file to define a config file that contains both arguments, and save it as `prepare_task_step3.py`. Note that defining a config file overwrites any command-line parameters:

In [ ]:
import argparse
import configparser

def main(number, other_number):
    result = number * other_number
    print(f'The result is {result}')
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-n1', type=int, help='A number', default=1)
    parser.add_argument('-n2', type=int, help='Another number', default=1)
    parser.add_argument('--config', '-c', type=argparse.FileType('r'), help='config file')
    args = parser.parse_args()
    
    if args.config:
        config = configparser.ConfigParser()
        config.read_file(args.config)
        # Transforming values into integers
        args.n1 = int(config['ARGUMENTS']['n1'])
        args.n2 = int(config['ARGUMENTS']['n2'])
    
    main(args.n1, args.n2)

3. Create the config file, config.ini. See the ARGUMENTS section and the `n1` and `n2` values:

4. Run teh command with the config file. Note that the config file overwrites the command-line parameters, as described in *step 2*:

`$python3 prepare_task_step3.py -c config.ini`

`The result is 35`

`$ python3 prepare_task_step3.py -c config.ini -n1 2 -n2 3`

`The resul is 35`

5. Add a parameter to store the result in a file, and save it as `prepare_task_step6.py`

In [ ]:
import argparse
import sys
import configparser

def main(number, other_number, output):
    result = number * other_number
    print(f'The result is {result}', file=output)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-n1', type=int, help='A number', default = 1)
    parser.add_argument('-n2', type=int, help='Another number', default=1)
    parser.add_argument('--config', '-c', type=argparse.FileType('r'), help='config file')
    parser.add_argument('-o', dest='output', type=argparse.FileType('w'), help='output file', default=sys.stdout)

    args = parser.parse_args()

    if args.config:
        config = configparser.ConfigParser()
        config.read_file(args.config)

        # Transforming values into integers
        args.n1 = int(config["ARGUMENTS"]["n1"])
        args.n2 = int(config["ARGUMENTS"]["n2"])
        
    main(args.n1, args.n2, args.output)

6. Run the result to check that it's sending the output to the defined file. Note that there's no output outside the result files:

```system
$ python3 prepare_task_step6.py -n1 3 -n2 5 -o result.txt
$ cat result.txt
The result is 15
$ python prepare_task_step6.py -c config.ini -0 result2.txt
$ cat result2.txt
The result is 35
```


### There's more...

In most cases, this configuration parser should be good enough, but if more power is needed, you can use YAML files as configuration files. YAML files are very common as configuration files. They are well struvtured and can be parsed directly, taking into account of various data types:

1. Add `PyYAML == 5.3`

2. Install the requirements in the virual environment:

`$ pip install -r requirements.txt`

3. Create the `prepare_task_taml.py` file:


In [ ]:
import yaml
import argparse
import sys


def main(number, other_number, output):
    result = number * other_number
    print(f"The result is {result}", file=output)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-n1", type=int, help="A number", default=1)
    parser.add_argument("-n2", type=int, help="Another number", default=1)
    parser.add_argument(
        "-c",
        dest="config",
        type=argparse.FileType("r"),
        help="config file in YAML format",
        default=None,
    )
    parser.add_argument(
        "-o",
        dest="output",
        type=argparse.FileType("w"),
        help="output file",
        default=sys.stdout,
    )
    args = parser.parse_args()

    if args.config:
        config = yaml.load(args.config, Loader=yaml.FullLoader)
        # No need to transform values
        args.n1 = config["ARGUMENTS"]["n1"]
        args.n2 = config["ARGUMENTS"]["n2"]

    main(args.n1, args.n2, args.output)

Note that the PyYAML yaml.load() function requires a *Loader* parameter. This is to avoid arbitrary code execution if the YAML file comes from an untrusted source. Always use yaml.SafeLoader unless you need a set of YAML language features. Never use loader other than yaml.SafeLoader if any part of the data coming from a YAML file vomes from an untrusted source (for example, user input). Refer to this articule for more information: https://github.com/yaml/pyyaml/wiki/PyYAML-yaml.
 load(input)-Deprecation.

4. Define the config file, config.yaml:

### Getting ready

We will produce a script, called cron.py:

In [ ]:
import argparse
import sys
from datetime import datetime, timezone
import configparser


def main(number, other_number, output):
    result = number * other_number
    print(
        f"[{datetime.now(timezone.utc).isoformat()}] The result is {result}",
        file=output,
    )


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    parser.add_argument(
        "--config",
        "-c",
        type=argparse.FileType("r"),
        help="config file",
        default="./config.ini",
    )
    parser.add_argument(
        "-o",
        dest="output",
        type=argparse.FileType("w"),
        help="output file",
        default=sys.stdout,
    )
    args = parser.parse_args()

    if args.config:
        config = configparser.ConfigParser()
        config.read_file(args.config)
        # Transforming values into integers
        args.n1 = int(config["ARGUMENTS"]["n1"])
        args.n2 = int(config["ARGUMENTS"]["n2"])

    main(args.n1, args.n2, args.output)

 Check that the task is producing the expected result and that you can log to a known 
file:
```
 $ python3 cron.py
 [2020-01-15 22:22:31.436912] The result is 35
 $ python3 cron.py -o /path/automate.log
 $ cat /path/automate.log
 [2020-01-15 22:28:08.833272] The result is 35
```


### Getting ready

As a starting point, we'll usea a task that will divide two numbers, as described in the command line.

This task is very similar to the one presented in *step 5* of How ti works for the *Preparing a task* recipe, earlier this chapter. However, instead of multiplying two number, we'll divide them.

#### How to do it...

1. Create the `task_with_error_handling_step1.py`

In [ ]:
import argparse
import sys


def main(number, other_number, output):
    result = number / other_number
    print(f"The result is {result}", file=output)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-n1", type=int, help="A number", default=1)
    parser.add_argument("-n2", type=int, help="Another number", default=1)
    parser.add_argument(
        "-o",
        dest="output",
        type=argparse.FileType("w"),
        help="output file",
        default=sys.stdout,
    )

    args = parser.parse_args()

    main(args.n1, args.n2, args.output)

2. Execute it a couple of times to see that it divides two numbers
3. check that dividing by 0 produces an error, and that the error is not logged on the result file
4. create the `task_with_error_handling_step4.py`

In [ ]:
import argparse
import sys
import logging

LOG_FORMAT = "%(asctime)s %(name)s %(levelname)s %(message)s"
LOG_LEVEL = logging.DEBUG


def main(number, other_number, output):
    logging.info(f"Dividing {number} between {other_number}")
    result = number / other_number
    print(f"The result is {result}", file=output)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-n1", type=int, help="A number", default=1)
    parser.add_argument("-n2", type=int, help="Another number", default=1)
    parser.add_argument(
        "-o",
        dest="output",
        type=argparse.FileType("w"),
        help="output file",
        default=sys.stdout,
    )
    parser.add_argument("-1", dest="log", type=str, help="log file", default=None)

    args = parser.parse_args()

    if args.log:
        logging.basicConfig(format=LOG_FORMAT, filename=args.log, level=LOG_LEVEL)
    else:
        logging.basicConfig(format=LOG_FORMAT, level=LOG_LEVEL)

    try:
        main(args.n1, args.n2, args.output)
    except Exception as exc:
        logging.exception("Error runing task")
        exit(1)

5. Run it to check that it displays the proper INFO and ERROR logs, and that it stores it on the log file:

### Sending email notifications

Email has become an inescapable tool for everyday use. It's arguably the best place to send a notification if an automated task has detected something. On the other hand, email inboces are already too full up with spam messages, so be careful.

Spam filters are also a reality. be careful with whom to send emails to and the number of emails to be sent. An email server or address can be labeled as a spam source, and all emails may be quietly dropped by the internet.

This recipe will show you how to send a single email using an existin email account. 

This approach is viable for spare emails sent to a couple of people, as a result of an automated task, but no more that that. Refer To *Chapter 0, Dealing with communication channels*, for more ideas on how to send emails, including groups.

### Getting ready

For this recipe, we require a valid email account set up, which includes the following:

- A valid email server using SMTP; SMTP in the standard email protocol
- A port to connect to
- An address
- A password

These four elements should be enough to be able to send an email.

Some emails services, for example, Gmail, will encourage you to set up 2FA, meaning that a password is not enough to send an email. Typically, they'll allow you to create a specific passeord for apps to use, bypassing the 2FA request. Check you email provider's information for options.

The email service you use should indicate what the SMTP server is and whay port to use in thei documentation. This can be retrieved from email clients as well, as they are the same parameters. Check your provider documentation. In the following example, we will use a Gmail account.

### How to do it..

1. Create the *email_task.py* file, as follows:

In [ ]:
import argparse
import configparser
import smtplib
from email.message import EmailMessage


def main(to_email, server, port, from_email, password):
    print(f"With love, from {from_email} to {to_email}")

    # Create the message
    subject = "With love, from ME to YOU"
    text = """This is an example test"""
    msg = EmailMessage()
    msg.set_content(text)
    msg["Subject"] = subject
    msg["From"] = from_email
    msg["To"] = to_email

    # Opent communication and send
    server = smtplib.SMTP_SSL(server, port)
    server.login(from_email, password)
    server.send_message(msg)
    server.quit()


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("email", type=str, help="destination email")
    parser.add_argument(
        "-c",
        dest="config",
        type=argparse.FileType("r"),
        help="config file",
        default=None,
    )
    args = parser.parse_args()

    if not args.config:
        print("Error, a config file is required")
        parser.print_help()
        exit(1)

    config = configparser.ConfigParser()
    config.read_file(args.config)

    main(
        args.email,
        server=config["DEFAULT"]["server"],
        port=config["DEFAULT"]["port"],
        from_email=config["DEFAULT"]["email"],
        password=config["DEFAULT"]["password"],
    )

2. Create a configuration file called *email_conf.ini* with the specifics of your email account. For example, for a Gmail account, fill in the following template. The template is available in GitHub at https://github.com/PacktPublishing/Python-Automation-Cookbook-Second-Edition/blob/
 master/Chapter02/email_conf.ini but be sure to fill it in with your data:

The email message content used in this recipe is very simple, but emails can be much more complicated than that.

The *To* fiel can contain multiple recipients. Separate them with commas, like this:

`message['To'] = ','.join(recipients)`

Emails can be defined in HTML, with an alternative plain text, and have attachments. The basic operations is to set up a *MIMEMultipart* and then attach each of the *MIME* parts that compose the email:

```
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

message = MIMEMultipart()
part1 = MIMEText('some text', 'plain')
message.attach(part1)
with open('path/image', 'rb') as image:
    part2 = MIMEImage(image.read())
message.attach(part2)
```

The most common SMTP connection is *SMTP_SSL*, which is more secure, as all communication with the server is encrypted and always requires a login and password. However, plain, unauthenticated SMTP exists - chec your email provider documentation.

Remember that this recipe is aimed toward simple notifications. Emails can grow quite complex if attaching different information. If your objective is an email for customers or any general group, try to use the ideas in *Chapter 9, Dealing with communication Channels*.